In [1]:
#import everything I need
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
class player: #class for players
    def __init__(self):
        self
class game: #each object - a game between two players
    def __init__(self):
        self
        
#Takes an integer number of a tournament (you can get it from the tournament url, and the numbers are consecutive) 
#Changes (creates) global list of all players in the tournament
#There are a couple tournaments on the site in another format, the function might not work for them (!!!)
def tournament(number): 
    global players
    #Parse the tournament page and create a table
    tournpage = requests.get('http://tt-liga.pro/tours/'+str(number))
    tournpage = BeautifulSoup(tournpage.text, 'html.parser')
    rows = tournpage.table.findAll('tr')[1:]
    table = [[cell.string.strip() for cell in row('td')] 
             for row in rows]
    players = [] #list of players
    a = [] #list of links to get players' ids
    for p in tournpage.table('a'):
        a.append(p['href'][8:]) #contains id of each player
    for i in range(len(table)):
        p = player() #create player object
        p.name = table[i][1] #add name
        p.rating = table[i][3] #add rating
        p.id = a[i] #add id
        players.append(p) 
#for two string player ids parses page with history of their games
def getplays(a,b):
    global page
    page = requests.get('http://tt-liga.pro/statistics/'+a+'?vs='+b)
    page = BeautifulSoup(page.text, 'html.parser')
#gets the data about previous games between all the players in the tournament    
def createplaystable():
    global players, page
    for i in range(len(players)):
        for j in range(len(players)):
            if j!=i: #I build two tables for each pair of players (numbers differ by sign)
                getplays(players[i].id, players[j].id)
                try:
                    rows = page('table')[0].findAll('tr')[1:-1]
                    table = [[cell.string for cell in row('td')[:-2]] for row in rows] #without the last two columns for now
                except: #occurs if there were no games between the players
                    table = 'None'
                try:
                    players[i].against.update({players[j].id: table})
                except: #occurs when players[i].against is not yet created
                    players[i].against = {players[j].id: table}
#takes players' ordinal numbers and returns a vector 'differences in sets factor' for all the previous games                  
def diffsetsfactor(n1, n2):
    global players
    dsf = []
    for a in players[n1].against[players[n2].id]:
        if a!='None':
            w = int(a[4][0]) #sets won
            l = int(a[4][-1]) #sets lost
            dsf.append(w-l)
    return(dsf)   
#creates game objects
def creategamesdata():
    global players, games
    games = [] #list of games
    for i in range(len(players)):
        for j in range(i+1,len(players)): #just one game for each pair of players
            g = game() #create game object
            g.id1 = players[i].id #id of the first player
            g.id2 = players[j].id #id of the second player
            g.dsf = diffsetsfactor(i,j) #vector of differences in sets in all the previous games between the players
            games.append(g) 

In [3]:
for i in [110, 236, 1101, 789, 634]: #check for some random tournaments
    tournament(i)
    createplaystable()
    creategamesdata()
    for g in games:
           print('{:4d} {:4s} {:4s} {:3d} {:+1.2f}'.format(i, g.id1, g.id2, len(g.dsf), np.mean(g.dsf)), g.dsf)

 110 80   57     2 +0.00 [-1, 1]
 110 80   568    2 -0.50 [-3, 2]
 110 80   550   13 -0.62 [-3, -3, -3, -3, 2, 2, 2, 1, -2, 2, 1, -2, -2]
 110 80   582    9 +0.11 [-3, -1, -3, 1, 2, 1, 3, -2, 3]
 110 57   568    2 +1.50 [2, 1]
 110 57   550    1 +3.00 [3]
 110 57   582    1 +2.00 [2]
 110 568  550    2 +1.00 [1, 1]
 110 568  582    4 -1.25 [-1, -1, -2, -1]
 110 550  582   11 +1.09 [3, -3, 1, 3, 1, 2, 2, 2, 2, -2, 1]
 236 588  216    8 +0.88 [2, 3, 1, 1, 1, -1, -1, 1]
 236 588  589   17 -0.29 [-1, -1, 2, 1, 2, -1, -2, -3, -2, 3, -2, 2, -1, 3, -3, -3, 1]
 236 588  584   10 -1.20 [-1, 0, -1, -2, -3, 1, -1, -2, -2, -1]
 236 588  40     5 +1.00 [1, 2, 3, -3, 2]
 236 216  589    1 +3.00 [3]
 236 216  584    6 -1.33 [-4, 1, -3, -2, 2, -2]
 236 216  40     2 +0.00 [2, -2]
 236 589  584    8 +0.50 [2, 1, 2, 2, 1, -2, -3, 1]
 236 589  40     2 +2.00 [2, 2]
 236 584  40     5 +1.80 [3, 3, 2, -2, 3]
1101 519  524   15 -0.27 [-3, 1, -1, -1, -1, -2, -3, 3, -1, 1, 1, -1, 1, -1, 3]
1101 519  554   27 

In [4]:
for i in [1148, 1149, 1150]: #check for tournaments tomorrow, July 13
    tournament(i)
    createplaystable()
    creategamesdata()
    for g in games:
        print('{:4d} {:4s} {:4s} {:3d} {:+1.2f}'.format(i, g.id1, g.id2, len(g.dsf), np.mean(g.dsf)), g.dsf)

1148 405  242    2 -1.50 [0, -3]
1148 405  314    1 +0.00 [0]
1148 405  258    1 +0.00 [0]
1148 242  314    1 +0.00 [0]
1148 242  258    1 +0.00 [0]
1148 314  258    1 +0.00 [0]
1149 97   605   11 +0.27 [0, 2, -3, -1, 1, -1, -2, 3, -1, 2, 3]
1149 97   791    5 +1.00 [0, -1, 3, 2, 1]
1149 97   985    5 -0.20 [0, 2, -2, -2, 1]
1149 97   653    1 +0.00 [0]
1149 605  791    4 +0.75 [0, 2, -1, 2]
1149 605  985   16 +0.56 [0, 3, -2, -3, 1, 3, 3, 1, -2, -3, 3, 2, 3, 1, 2, -3]
1149 605  653    1 +0.00 [0]
1149 791  985    3 -1.67 [0, -3, -2]
1149 791  653    1 +0.00 [0]
1149 985  653    1 +0.00 [0]
1150 139  651    6 -0.50 [0, -2, -3, -3, 3, 2]
1150 139  564   12 +1.17 [0, -1, 3, 1, 3, 2, -1, -2, 2, 3, 2, 2]
1150 139  693    8 -0.75 [0, 1, -3, 1, 3, -2, -3, -3]
1150 139  559    9 +0.67 [0, 2, 2, -1, 2, -3, 3, -2, 3]
1150 651  564   24 -0.33 [0, 0, -1, -1, 1, -2, 2, 1, -2, 2, -3, 1, -2, -1, 1, -2, 1, -1, 1, -1, -3, -2, 1, 2]
1150 651  693    8 +0.88 [0, 1, -1, 3, 2, -1, 1, 2]
1150 651  559   14